In [2]:
import os
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename
from PIL import Image
import torchvision.transforms as T
import torchvision
import numpy as np


In [3]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
       

In [5]:

UPLOAD_FOLDER = '/photo'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

app = Flask(__name__)
cnt = 0
path = ""
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)

        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        # print(file.read())

        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)

        if file and allowed_file(file.filename):
            # filename = file.filename
            # file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            fl = open('static/' + str(0) + '.' + file.filename.rsplit('.', 1)[1].lower(), 'wb+')
            fl.write(file.read())
            fl.close()


            im = Image.open('static/' + str(0) + '.' + file.filename.rsplit('.', 1)[1].lower())
    
            a = np.array(im)
            t = T.Compose([T.ToTensor()])(im)
            if t.size()[0] > 3:
                t = t[0:3]
            c = []
            c.append(t)
            
            res: [{torch.tensor}]
            res = model(c)

            humanScore: float = 0.0
            ind = -1
            for key, t in res[0].items():
                if (key == 'labels'):
                    for i in range(len(t)):
                        if (t[i] == 1):
                            ind = i
                if (key == 'scores' and ind != -1):
                    humanScore = t[ind]
                    print(t[ind])

            if (humanScore > 0.75):
                path = '../static/' + str(0) + '.' + file.filename.rsplit('.', 1)[1].lower()
                return render_template('ok.html', name_file= path)
            else:
                return redirect(url_for('no'))
        else:
            flash('Error')
            return redirect(request.url)
    return render_template('base.html')


from flask import send_from_directory

@app.route('/ok')
def ok():
    return render_template('ok.html', name_file= lol)

@app.route('/no')
def no():
    return render_template('no.html')


if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Dec/2019 10:35:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2019 10:35:38] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [23/Dec/2019 10:35:38] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [23/Dec/2019 10:35:38] "GET /no HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2019 10:35:45] "GET / HTTP/1.1" 200 -
